In [2]:
import scipy
import pandas as pd
import numpy as np
import itertools

In [50]:
#initial solution
def initial_solution(A,b, init_basis=None):
    A = np.array(A)
    b=np.array(b)
    if np.linalg.matrix_rank(A)<len(b):
        print('Ax=b is not solvable')
    if init_basis!=None:
        A_inv = np.linalg.inv(A[np.ix_(range(len(b)), init_basis)])
        in_the_basis = init_basis
        table = np.matmul(A_inv, A)
        b_hat = np.matmul(A_inv, b)
    else:
        s=0
        ind = -1
        combinations = [list(comb) for comb in itertools.combinations(range(A.shape[1]), A.shape[0])]
        while s<len(b):
            ind += 1
            s = np.linalg.matrix_rank(A[np.ix_(range(len(b)), combinations[ind])])
        A_inv = np.linalg.inv(A[np.ix_(range(len(b)), combinations[ind])])
        in_the_basis = combinations[ind]
        table = np.matmul(A_inv, A)
        b_hat = np.matmul(A_inv, b)
    return(in_the_basis, table, b_hat)
            
            
    
    

In [228]:
def initial_solution2(A,b, init_basis=None):
    A_hat = A
    b_hat = b
    in_the_basis = range(A.shape[1],A.shape[1]+A.shape[0])
    in_the_basis = np.array(in_the_basis)
    kakao = (in_the_basis>=A.shape[1])
    if init_basis == None:
        for i in np.where(kakao)[0]:
            if (np.all(A_hat[i]==0)) &(b_hat[i]==0):
                A_hat = np.delete(A_hat, i,0)
                b_hat = np.delete(b_hat,i)
                in_the_basis = np.delete(in_the_basis,i)
            else:

                ind = (A_hat[i]!=0).argmax(axis=0)
                A_hat, b_hat = pivot(A_hat,b_hat,i,ind)
                in_the_basis[i]=ind
                #print(i, ind)
    if init_basis!=None:
        init_basis = list(init_basis)
        for i in np.where(kakao)[0]:
            if (np.all(A_hat[i]==0)) &(b_hat[i]==0):
                A_hat = np.delete(A_hat, i,0)
                b_hat = np.delete(b_hat,i)
                in_the_basis = np.delete(in_the_basis,i)
            else:
                ind = [j for j in init_basis if A_hat[i][j]!=0][0]
                init_basis.remove(ind)
                A_hat, b_hat = pivot(A_hat,b_hat,i,ind)
                in_the_basis[i]=ind
                #print(i, ind)
    return(in_the_basis, A_hat, b_hat)


In [51]:
#pivot
def pivot(A, b,index1, index2):
    P = np.identity(A.shape[0])
    v= -A[:, index2]/A[index1, index2]
    v[index1] = 1/A[index1, index2]
    P[:, index1] = v
    new_A = np.matmul(P, A)
    new_b_hat = np.matmul(P, b)
    return(new_A, new_b_hat)
    
    

In [52]:
#creates a pandas dataframe of the solution
def nice_table(table,b_hat=None,init_basis=None):
    tableau = pd.DataFrame(table, columns = ['x'+str(i) for i in range(table.shape[1])])
    tableau['b'] = b_hat
    if np.all(init_basis) != None:
        tableau.index = ['x'+str(i) for i in init_basis]
    return(tableau)
    

In [230]:
def MBU_simplex(A,b, init_basis = None):
    #initial solution
    init_basis, table, b_hat = initial_solution2(A,b, init_basis)
    print('initial solution:', '\n' ,nice_table(table, b_hat, init_basis))
    end_while= np.all(np.array(b_hat)>=0)
    no_print=False
    countr_outer = 0
    while end_while==False:
        countr_outer+=1
        #set of indices where b_hat<0
        index_minus_orig  = np.array(init_basis)[np.where(np.array(b_hat)<0)]
        index_minus = np.array(range(A.shape[0]))[np.where(np.array(b_hat)<0)]
        #indices, where b_hat>0
        #index_plus_orig  = np.array(init_basis)[np.where(np.array(b_hat)>0)]
        #index_plus = np.array(range(A.shape[0]))[np.where(np.array(b_hat)>0)]
        r_original = np.min(index_minus_orig)
        r = index_minus[np.argmin(index_minus_orig)]
        #print(r)
        rDone = False
        #print(np.where(table[r, :]<0)[0])
        countr_inner = 0
        print('r=', 'x'+str(r_original))
        while rDone==False:
            
            if list(np.where(table[r, :]<0)[0])==[]:
                print('no feasible solution!')
                no_print = True
                rDone=True
                end_while=True
            else:
                J_r_=np.where(table[r, :]<0)
                J_r_original = np.array(range(table.shape[1]))[J_r_]
                s = np.min(J_r_)
                s_original = range(table.shape[1])[s]
                theta1 = b_hat[r]/table[r,s]
                print('s=', 'x'+str(s_original))

                good_row = table[:,s][np.where((table[:,s]>0) & (b_hat>=0))]

                good_b = b_hat[np.where((table[:,s]>0) & (b_hat>=0))]
                
                good_index = np.where((table[:,s]>0) & (b_hat>=0))
                #print(good_b,good_index, good_index[0])
                if list(good_b)==[]:
                    theta2 = np.inf
                else:
                    theta2 = np.min(good_b/good_row)
                    q = good_index[0][np.argmin(good_b/good_row)]
                    #q_orig = range(table.shape[1])[q]
                    q_orig = init_basis[q]
                print('theta2 =', theta2, ', theta1 =', theta1)
                
                if theta1<=theta2:
                    print('theta2>= theta1 => pivot on (r,s)=', '(','x'+str(r_original),',','x'+str(s_original), ')')
                    table, b_hat = pivot(np.array(table), np.array(b_hat), r,s)
                    rDone=True
                    init_basis[r] = s_original
                    print('new basis: ', ['x'+str(i) for i in init_basis])
                    countr_inner +=1
                    print('iterations: outer:', str(countr_outer), ' inner:', str(countr_inner))
                    print('_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ ')
                if theta2<theta1:
                    print('theta2< theta1 => pivot on (q,s)=', '(','x'+str(q_orig),',','x'+str(s_original), ')')
                    table, b_hat = pivot(np.array(table), np.array(b_hat), q,s)
                    init_basis[q] = s_original
                    print('new basis: ', ['x'+str(i) for i in init_basis])
                    countr_inner +=1
                    print('iterations: outer:', str(countr_outer), ' inner:', str(countr_inner))
                    print('. . . . . . . . . . . . . . . . . . . . . . .')
                print(nice_table(table, b_hat, init_basis))
                #print('b_hat', b_hat)
                
                end_while = np.all(np.array(b_hat)>=0)
                #print(end_while)
    if no_print == False:
        solution = np.zeros((1,A.shape[1]))[0]
        solution[init_basis]=b_hat
        print('______________________________________________')
        print('x =',solution,'\n', 'check:', '\n',  nice_table(A, b), '\n','A*x = ', np.matmul(A, solution))
        return(solution)
    else:
        return('no feasible solution!')
                    
                    
    

# examples

## first example

In [231]:
A = np.array([[1,0,0], [0,1,0], [0,0,-1]])
b=np.array([1,2,3])

In [232]:
MBU_simplex(A,b)

initial solution: 
      x0   x1   x2    b
x0  1.0  0.0  0.0  1.0
x1  0.0  1.0  0.0  2.0
x2  0.0  0.0  1.0 -3.0
r= x2
no feasible solution!


'no feasible solution!'

## second example

In [233]:
A=np.array([[1,2,0,-1,1,1,0],[1,0,2,1,1,0,0], [0,2,0,0,1,0,-1],[0,2,0,0,1,0,-1]])
b = np.array([30,80,40, 40])

In [234]:
MBU_simplex(A,b)

initial solution: 
      x0   x1   x2   x3   x4   x5   x6     b
x0  1.0  0.0  0.0 -1.0  0.0  1.0  1.0 -10.0
x1  0.0  1.0  0.0  0.0  0.5  0.0 -0.5  20.0
x2  0.0  0.0  1.0  1.0  0.5 -0.5 -0.5  45.0
r= x0
s= x3
theta2 = 45.0 , theta1 = 10.0
theta2>= theta1 => pivot on (r,s)= ( x0 , x3 )
new basis:  ['x3', 'x1', 'x2']
iterations: outer: 1  inner: 1
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
     x0   x1   x2   x3   x4   x5   x6     b
x3 -1.0  0.0  0.0  1.0  0.0 -1.0 -1.0  10.0
x1  0.0  1.0  0.0  0.0  0.5  0.0 -0.5  20.0
x2  1.0  0.0  1.0  0.0  0.5  0.5  0.5  35.0
______________________________________________
x = [ 0. 20. 35. 10.  0.  0.  0.] 
 check: 
    x0  x1  x2  x3  x4  x5  x6   b
0   1   2   0  -1   1   1   0  30
1   1   0   2   1   1   0   0  80
2   0   2   0   0   1   0  -1  40
3   0   2   0   0   1   0  -1  40 
 A*x =  [30. 80. 40. 40.]


array([ 0., 20., 35., 10.,  0.,  0.,  0.])

### with given initial solution

In [235]:
MBU_simplex(A,b, [5,2,6])

initial solution: 
      x0   x1   x2   x3   x4   x5   x6     b
x5  1.0  2.0  0.0 -1.0  1.0  1.0  0.0  30.0
x2  0.5  0.0  1.0  0.5  0.5  0.0  0.0  40.0
x6  0.0 -2.0  0.0  0.0 -1.0  0.0  1.0 -40.0
r= x6
s= x1
theta2 = 15.0 , theta1 = 20.0
theta2< theta1 => pivot on (q,s)= ( x5 , x1 )
new basis:  ['x1', 'x2', 'x6']
iterations: outer: 1  inner: 1
. . . . . . . . . . . . . . . . . . . . . . .
     x0   x1   x2   x3   x4   x5   x6     b
x1  0.5  1.0  0.0 -0.5  0.5  0.5  0.0  15.0
x2  0.5  0.0  1.0  0.5  0.5  0.0  0.0  40.0
x6  1.0  0.0  0.0 -1.0  0.0  1.0  1.0 -10.0
s= x3
theta2 = 80.0 , theta1 = 10.0
theta2>= theta1 => pivot on (r,s)= ( x6 , x3 )
new basis:  ['x1', 'x2', 'x3']
iterations: outer: 1  inner: 2
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
     x0   x1   x2   x3   x4   x5   x6     b
x1  0.0  1.0  0.0  0.0  0.5  0.0 -0.5  20.0
x2  1.0  0.0  1.0  0.0  0.5  0.5  0.5  35.0
x3 -1.0  0.0  0.0  1.0  0.0 -1.0 -1.0  10.0
______________________________________________
x = [ 0. 20. 35. 

array([ 0., 20., 35., 10.,  0.,  0.,  0.])

In [236]:
A=np.array([[-1,0,-1,1,1,0,0], [-1, -1, -2, -3, 0, 1, 0], [1, 0, 0, -2, 0, 0, 1]])
b=np.array([-2,-2, -3])

In [237]:
MBU_simplex(A,b)

initial solution: 
      x0   x1   x2   x3   x4   x5   x6    b
x0  1.0  0.0  0.0 -2.0  0.0  0.0  1.0 -3.0
x1  0.0  1.0  0.0  3.0  2.0 -1.0  1.0 -5.0
x2  0.0  0.0  1.0  1.0 -1.0  0.0 -1.0  5.0
r= x0
s= x3
theta2 = 5.0 , theta1 = 1.5
theta2>= theta1 => pivot on (r,s)= ( x0 , x3 )
new basis:  ['x3', 'x1', 'x2']
iterations: outer: 1  inner: 1
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
     x0   x1   x2   x3   x4   x5   x6    b
x3 -0.5  0.0  0.0  1.0  0.0  0.0 -0.5  1.5
x1  1.5  1.0  0.0  0.0  2.0 -1.0  2.5 -9.5
x2  0.5  0.0  1.0  0.0 -1.0  0.0 -0.5  3.5
r= x1
s= x5
theta2 = inf , theta1 = 9.5
theta2>= theta1 => pivot on (r,s)= ( x1 , x5 )
new basis:  ['x3', 'x5', 'x2']
iterations: outer: 2  inner: 1
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
     x0   x1   x2   x3   x4   x5   x6    b
x3 -0.5  0.0  0.0  1.0  0.0  0.0 -0.5  1.5
x5 -1.5 -1.0  0.0  0.0 -2.0  1.0 -2.5  9.5
x2  0.5  0.0  1.0  0.0 -1.0  0.0 -0.5  3.5
______________________________________________
x = [0.  0.  3.5 1.5 0.  

array([0. , 0. , 3.5, 1.5, 0. , 9.5, 0. ])

In [238]:
A=np.array([[1,-2,1,0,0], [0,0,0,1,0], [-4,0,0,0,1]])
b=np.array([0,1,-2])

In [239]:
MBU_simplex(A,b, [2,3,4])

initial solution: 
      x0   x1   x2   x3   x4    b
x2  1.0 -2.0  1.0  0.0  0.0  0.0
x3  0.0  0.0  0.0  1.0  0.0  1.0
x4 -4.0  0.0  0.0  0.0  1.0 -2.0
r= x4
s= x0
theta2 = 0.0 , theta1 = 0.5
theta2< theta1 => pivot on (q,s)= ( x2 , x0 )
new basis:  ['x0', 'x3', 'x4']
iterations: outer: 1  inner: 1
. . . . . . . . . . . . . . . . . . . . . . .
     x0   x1   x2   x3   x4    b
x0  1.0 -2.0  1.0  0.0  0.0  0.0
x3  0.0  0.0  0.0  1.0  0.0  1.0
x4  0.0 -8.0  4.0  0.0  1.0 -2.0
s= x1
theta2 = inf , theta1 = 0.25
theta2>= theta1 => pivot on (r,s)= ( x4 , x1 )
new basis:  ['x0', 'x3', 'x1']
iterations: outer: 1  inner: 2
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
     x0   x1   x2   x3     x4     b
x0  1.0  0.0  0.0  0.0 -0.250  0.50
x3  0.0  0.0  0.0  1.0  0.000  1.00
x1  0.0  1.0 -0.5  0.0 -0.125  0.25
______________________________________________
x = [0.5  0.25 0.   1.   0.  ] 
 check: 
    x0  x1  x2  x3  x4  b
0   1  -2   1   0   0  0
1   0   0   0   1   0  1
2  -4   0   0   0   1 -

array([0.5 , 0.25, 0.  , 1.  , 0.  ])

______________________________________________________

In [22]:
akk = ['x1','x2','x3']

In [23]:
['x'+str(i) for i in range(A.shape[1])]

['x0', 'x1', 'x2', 'x3', 'x4']

In [269]:
[A,np.array(b).T ,np.array(akk).T]

[array([[ 1,  2,  0, -1,  1,  1,  0],
        [ 1,  0,  2,  1,  1,  0,  0],
        [ 0,  2,  0,  0,  1,  0, -1]]), array([30, 80, 40]), array([1, 2, 3])]

In [273]:
np.concatenate([A.T,b ])

ValueError: all the input arrays must have same number of dimensions

array([[ 1,  1,  0],
       [ 2,  0,  2],
       [ 0,  2,  0],
       [-1,  1,  0],
       [ 1,  1,  1],
       [ 1,  0,  0],
       [ 0,  0, -1]])

In [278]:
makako = pd.DataFrame(A, columns = ['x'+str(i) for i in range(A.shape[1])])

In [279]:
makako['b'] = b

In [287]:
makako.index = akk

In [289]:
print(makako)

    x0  x1  x2  x3  x4  x5  x6   b
x1   1   2   0  -1   1   1   0  30
x2   1   0   2   1   1   0   0  80
x3   0   2   0   0   1   0  -1  40


In [189]:
b_hat = [ 30. , 40., -40.]

In [153]:
np.array(range(A.shape[0]))[

2

In [127]:
[np.where(np.array([b_hat])<0)]

NameError: name 'b_hat' is not defined

In [112]:
np.where(kalap==1)

(array([0], dtype=int64),)

In [106]:
kalap = np.array([1,2,3])
kabat = np.array([1,2,3])


In [110]:
np.all(np.array([1,2,3])>=0)

True

In [107]:
kalap/kabat

array([1., 1., 1.])

In [101]:
A[1, :]

array([1, 0, 2, 1, 1, 0, 0])

In [105]:
np.where(np.array([-1,-2,4,5,-7,3,9])<0)

(array([0, 1, 4], dtype=int64),)

In [94]:
-A[:,1]/4

array([-0.5,  0. , -0.5])

In [51]:
np.where(np.array([5,-2,-4,5,7,-8])<0)

(array([1, 2, 5], dtype=int64),)

In [55]:
np.argmin(np.array([7,8,9,10,11,12])[np.where(np.array([5,-2,-4,5,7,-8])<0)])

0

In [66]:
roka, kakao, makao, a_inv = initial_solution(A,b, [5,2,6])

In [64]:
roka

[5, 2, 6]

In [194]:
kakao

array([[ 1. ,  2. ,  0. , -1. ,  1. ,  1. ,  0. ],
       [ 0.5,  0. ,  1. ,  0.5,  0.5,  0. ,  0. ],
       [ 0. , -2. ,  0. ,  0. , -1. ,  0. ,  1. ]])

In [191]:
alma, korte = pivot(kakao, makao, 0,1)

In [192]:
alma

array([[ 0.5,  1. ,  0. , -0.5,  0.5,  0.5,  0. ],
       [ 0.5,  0. ,  1. ,  0.5,  0.5,  0. ,  0. ],
       [ 1. ,  0. ,  0. , -1. ,  0. ,  1. ,  1. ]])

In [193]:
korte

array([ 15.,  40., -10.])

In [76]:
P = np.array([[1/2,0,0], [0,1,0], [1,0,1]])

In [77]:
np.matmul(P, kakao)

array([[ 0.5,  1. ,  0. , -0.5,  0.5,  0.5,  0. ],
       [ 0.5,  0. ,  1. ,  0.5,  0.5,  0. ,  0. ],
       [ 1. ,  0. ,  0. , -1. ,  0. ,  1. ,  1. ]])

In [90]:
np.matmul(P, makao)

array([ 15.,  40., -10.])

In [86]:
ido = np.identity(A.shape[0])

In [87]:
ido

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [88]:
ido[:, 0] = [1/2, 0, 1]

In [89]:
ido

array([[0.5, 0. , 0. ],
       [0. , 1. , 0. ],
       [1. , 0. , 1. ]])

In [62]:
np.linalg.inv()

LinAlgError: Last 2 dimensions of the array must be square

In [23]:
A.shape

(3, 7)

In [389]:
A=np.array([[1,2,0,-1,1,1,0],[1,0,2,1,1,0,0], [0,2,0,0,1,0,-1]])

In [390]:
b = np.array([30,80,40])

In [7]:
np.linalg.matrix_rank(A)

3

In [14]:
A[np.ix_([0,1,2], [0,2])]

array([[1, 0],
       [0, 0],
       [0, 1]])

In [17]:
import itertools
for comb in itertools.combinations([0,1, 2, 3, 4], 3):
    print(list(comb))

[0, 1, 2]
[0, 1, 3]
[0, 1, 4]
[0, 2, 3]
[0, 2, 4]
[0, 3, 4]
[1, 2, 3]
[1, 2, 4]
[1, 3, 4]
[2, 3, 4]


In [360]:
np.concatenate([[4*i+1, 4*i+2] for i in range(21)])

array([ 1,  2,  5,  6,  9, 10, 13, 14, 17, 18, 21, 22, 25, 26, 29, 30, 33,
       34, 37, 38, 41, 42, 45, 46, 49, 50, 53, 54, 57, 58, 61, 62, 65, 66,
       69, 70, 73, 74, 77, 78, 81, 82])

In [361]:
np.concatenate([[4*i+3, 4*i+4] for i in range(21)])

array([ 3,  4,  7,  8, 11, 12, 15, 16, 19, 20, 23, 24, 27, 28, 31, 32, 35,
       36, 39, 40, 43, 44, 47, 48, 51, 52, 55, 56, 59, 60, 63, 64, 67, 68,
       71, 72, 75, 76, 79, 80, 83, 84])

In [ ]:
19, 20, 23, 24, 27, 28, 31, 32

In [ ]:
35, 36, 39, 40, 43, 44, 47, 48

In [ ]:
51, 52, 55, 56, 59, 60, 63, 64

In [ ]:
67, 68, 71, 72, 75, 76, 79, 80 

In [3]:
A=np.array([[1,2,0,-1,1,1,0],[1,0,2,1,1,0,0], [0,2,0,0,1,0,-1]])

In [61]:
U,s,V = np.linalg.svd(A)

In [62]:
U

array([[-5.94935890e-01,  1.07717292e-01,  7.96522612e-01,
         4.84613418e-17],
       [-1.60735547e-01, -9.86906438e-01,  1.34077081e-02,
        -6.88006894e-17],
       [-5.56873132e-01,  8.48901286e-02, -4.27417806e-01,
        -7.07106781e-01],
       [-5.56873132e-01,  8.48901286e-02, -4.27417806e-01,
         7.07106781e-01]])

In [63]:
s

array([4.19885624e+00, 2.59206883e+00, 1.62812329e+00, 1.94729607e-16])

In [64]:
V

array([[-0.17997078, -0.81387981, -0.07656159,  0.10340919, -0.4452207 ,
        -0.14168999,  0.26524992],
       [-0.33918434,  0.21411279, -0.76148166, -0.42229732, -0.27368443,
         0.04155649, -0.0654999 ],
       [ 0.49746252, -0.07163217,  0.01647014, -0.48099238, -0.02758101,
         0.48922745,  0.52504354],
       [-0.60556081,  0.31022157,  0.54698639, -0.17899693, -0.30941505,
         0.11553579,  0.31102809],
       [-0.43741557, -0.29074733, -0.06508732, -0.22357087,  0.79116108,
         0.00417829,  0.20966641],
       [-0.21673045, -0.09018292, -0.11726053,  0.44590233,  0.00534917,
         0.83764944, -0.17501666],
       [ 0.00933671,  0.31260168, -0.31117878,  0.5465941 ,  0.06642675,
        -0.15437272,  0.69163012]])

In [69]:
A

array([[ 1,  2,  0, -1,  1,  1,  0],
       [ 1,  0,  2,  1,  1,  0,  0],
       [ 0,  2,  0,  0,  1,  0, -1],
       [ 0,  2,  0,  0,  1,  0, -1]])

In [67]:
A_inv = np.linalg.pinv(A)

In [68]:
np.matmul(A_inv, A)

array([[ 0.39490446,  0.03821656,  0.28025478, -0.11464968,  0.15923567,
         0.25477707,  0.23566879],
       [ 0.03821656,  0.7133758 , -0.10191083, -0.14012739,  0.30573248,
         0.08917197, -0.26751592],
       [ 0.28025478, -0.10191083,  0.58598726,  0.30573248,  0.24203822,
        -0.01273885,  0.03821656],
       [-0.11464968, -0.14012739,  0.30573248,  0.42038217,  0.08280255,
        -0.26751592, -0.19745223],
       [ 0.15923567,  0.30573248,  0.24203822,  0.08280255,  0.27388535,
         0.03821656, -0.11464968],
       [ 0.25477707,  0.08917197, -0.01273885, -0.26751592,  0.03821656,
         0.2611465 ,  0.21656051],
       [ 0.23566879, -0.26751592,  0.03821656, -0.19745223, -0.11464968,
         0.21656051,  0.35031847]])

In [70]:
np.matmul(A_inv, b)

array([ 9.42675159,  9.29936306, 21.52866242, 12.10191083, 15.41401274,
       -1.33757962, -5.98726115])

In [18]:
V

array([[-0.28224801, -0.77234698, -0.1414097 ,  0.14083831, -0.45687834,
        -0.21154316,  0.17463033],
       [-0.29369846,  0.26952278, -0.75119417, -0.4574957 , -0.24083569,
         0.08189862, -0.05286277],
       [-0.47852036,  0.21026986, -0.04120534,  0.43731501,  0.08453226,
        -0.45791768, -0.56305262],
       [ 0.3129932 ,  0.14784123, -0.48852295,  0.73846351, -0.07441081,
         0.20419867,  0.22127164],
       [-0.22842345, -0.32769259, -0.30068946, -0.01482756,  0.84462994,
         0.02435113,  0.18924476],
       [-0.49810782, -0.04044152,  0.19994336,  0.16485995, -0.06663884,
         0.81048966, -0.14752189],
       [-0.45479899,  0.39465076,  0.21205322,  0.08301097, -0.05231842,
        -0.19917314,  0.73698311]])

In [19]:
len(s)

3

In [46]:
np.linalg.matrix_rank(np.c_[A, b])

3

In [47]:
np.linalg.matrix_rank(A)

3

In [150]:
A=np.array([[1,2,0,-1,1,1,0],[1,0,2,1,1,0,0], [0,2,0,0,1,0,-1], [0,2,0,0,1,0,-1]])
b = np.array([30,80,40,40])

In [78]:
A_left = np.matmul(A.T,np.linalg.inv(np.matmul(A,A.T)))

In [80]:
np.matmul(A_left,b)

array([ 9.42675159,  9.29936306, 21.52866242, 12.10191083, 15.41401274,
       -1.33757962, -5.98726115])

In [127]:
A[2]

array([ 0,  2,  0,  0,  1,  0, -1])

In [183]:
init_basis = range(A.shape[1],A.shape[1]+A.shape[0])

In [184]:
init_basis = np.array(init_basis)

In [185]:
np.any(init_basis>=A.shape[1])

True

In [186]:
kakao = (init_basis>=A.shape[1])

In [187]:
for i in np.where(kakao)[0]:
    print(i)

0
1
2
3


In [188]:
from copy import *

In [189]:
A_hat = A
b_hat = b
in_the_basis = range(A.shape[1],A.shape[1]+A.shape[0])
in_the_basis = np.array(in_the_basis)
kakao = (in_the_basis>=A.shape[1])
for i in np.where(kakao)[0]:
    if (np.all(A_hat[i]==0)) &(b_hat[i]==0):
        A_hat = np.delete(A_hat, i,0)
        b_hat = np.delete(b_hat,i)
        in_the_basis = np.delete(in_the_basis,i)
    else:
        print(i, ind)
        ind = (A_hat[i]!=0).argmax(axis=0)
        A_hat, b_hat = pivot(A_hat,b_hat,i,ind)
        in_the_basis[i]=ind

0 1
1 0
2 1


In [201]:
init_basis=np.array([5,4,2])

In [205]:
[j for j in init_basis if A[i][j]!=0][0]

4

In [204]:
A[i]

array([ 0,  2,  0,  0,  1,  0, -1])

In [168]:
A_hat

array([[ 1. ,  0. ,  0. , -1. ,  0. ,  1. ,  1. ],
       [ 0. ,  1. ,  0. ,  0. ,  0.5,  0. , -0.5],
       [ 0. ,  0. ,  1. ,  1. ,  0.5, -0.5, -0.5]])

In [173]:
b_hat = np.delete(b_hat, i)

In [116]:
A_hat

array([[ 1,  2,  0, -1,  1,  1,  0],
       [ 1,  0,  2,  1,  1,  0,  0],
       [ 0,  2,  0,  0,  1,  0, -1],
       [ 0,  2,  0,  0,  1,  0, -1]])

In [222]:
kakao =[4,5,6]

In [223]:
kakao.remove(4)

In [224]:
kakao

[5, 6]

In [ ]:
if np.linalg.matrix_rank(np.c_[A, b])>np.linalg.matrix_rank(A):
    return ('Ax=b is not solvable')
else:
    while np.any(init_basis>=A.shape[1]):
        
    

In [242]:
pd.__version__

'1.1.3'

In [243]:
np.__version__

'1.16.4'

In [244]:
scipy.__version__

'1.2.1'